In [ ]:
# solar viability rating alg
# median income by block -> can we make more granular? 
# research amt of solar energy (then the hours+roof space to produce it) needed to offset gas usage

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math


In [3]:
solar_data = pd.read_csv('solar_data.csv')
census_data = pd.read_csv('census_data.csv')
gas_meter_data = pd.read_csv('gas_meter_data.csv')

In [ ]:
# we have a list of the house_ids that are using gas heating
    #solar csv has sq_ft_of_roof_space which we can map to from the house id
# a function that maps house_id to the block
    # the block data has the median income

# function:
# for each house_id in the list of gas-heating houses map to their block using the function
# store their block
# retrieve block's median income from census_data.csv
# for each house_id in the list of gas-heating houses map to solar_data.csv to get avaliable square footage 
# put median_income and square_footage into the golden ratio function
# if no continue

# else append house_ids block # to a list of blocks 
# sort the list by number of times blocks are mentioned
# graph the blocks by how many houses each block has that passes the golden ratio
    # pick a threshold for how many houses is signifacant enough to put into a list of blocks we reccomend advertising solar energy to

In [6]:
#FUNCTION: Maps house to block
blocks_and_house_ids = census_data[['Block','house_ids']]
solar_house_ids = solar_data['house_id']
house_id_to_block = {}

for solar_row in range(len (solar_data. index)) :
    for census_row in range (len (census_data.index)) :
        house_ids_as_list = blocks_and_house_ids ['house_ids']. loc[census_row]
        solar_house_id = solar_house_ids.loc [solar_row]
        if solar_house_id in house_ids_as_list:
            house_id_to_block.update({solar_house_id:blocks_and_house_ids ['Block'].loc[census_row]})

print(house_id_to_block.get("67741535"))   #prints block house is in

811900-1


In [7]:
# find out if gas meter id attribute is not null in solar_data. if not, note down house id. these house ids are using gas

gas_house_ids = []


for solar_row in range(len(solar_data.index)):
       if math.isnan(solar_data['gas_meter_id'].loc[solar_row]):
               continue
       else:
            gas_house_ids.append(solar_data['house_id'].loc[solar_row])  


print(gas_house_ids)

# care about these houses - from these, select good candidates for solar
# next: aggregate gas meter readings to use when comparing with what solar can generate, to see if switching to solar is good idea

['a90ea0b3', '049bf03c', '569aaf4b', 'ac2f7c77', '7407e44b', '8c46fac9', '5edb8c2b', '0a4bda96', 'efd90c76', '399ac66c', '72be14aa', '7a94fe5b', '17013f80', '18f1daf5', '4ca0c4ec', 'f21423f0', 'a99e365d', 'c8d2352d', '6c40ece7', '6e369e62', 'b0d910d9', '44f1b9f3', '5c229c2c', 'cdaaab25', '20efba60', 'bed617be', '37a5ae24', 'a843fe8a', '55aad0b1', '000fe7ba', '2cf6190e', '0d12f2a9', 'c73e96d2', 'e0141bf2', '314fdb11', 'dec094c1', '53c0a6be', '64910021', '5977176c', '445a5d52', '1a0e7a9f', 'c10abddb', 'ae1d9225', 'be2d2305', 'c5988d9b', 'b590e55f', '50ee1f40', '79876464', '01699cd1', 'bf66339e', '57ab0b40', 'cf1f0e42', 'd5916302', '376d88fb', '2eddf669', '66b9f47a', 'd0e8416b', '88194881', '68804a61', '135afd08', 'd09c7764', 'c109f9ff', 'ba141335', 'b58c0ce3', 'a602dc54', 'ebb6aac2', 'f49a40a7', 'e48445a5', '4aae9cc0', 'c6135eac', '4ad69f26', '50d1ca84', '3d870d2d', 'e69c77c6', 'e8bb8c9f', 'c4ea6ba3', '441a2a56', '9b089c80', 'c038a5f0', 'a708c4b7', 'c296901c', '342b3bfc', 'a6787838', 'd4

In [53]:
def goldenratio (median_income: int, sq_ft_avaliable: int, total_gas_usage_per_year: int) -> bool:
    #10% of income should go to heating
    #a single panel generates 2kWh a day
    total_solar_panels = math.floor(sq_ft_avaliable/15) #rounds down to the closest integer, 3ft by 5ft panels
    energy_generated_per_year = total_solar_panels * 2 * 365 #2kWh a day for 265 days

    if total_gas_usage_per_year > energy_generated_per_year:
        return False
    
    else: 
        cost_ratio = round(median_income/(total_solar_panels * 15 * 5)) #$5 per squarefoot of solar panels
        if cost_ratio > 3: #if cost_ratio is greater than 30% of median income the solar switch isnt feasible
            return False
        else:
            return True

In [50]:
def median_income_retrieval (house_id_to_find: str) -> int:
    Median_Income_available = 0
    # retrieves house_id's block number
    block = house_id_to_block.get(house_id_to_find)
    #finds row the Block is listed in the census_csv
    selected_row_in_census_data = census_data[census_data['Block'] == block] 
    # if the row isn't empty we retrieve the Block's median income
    if not selected_row_in_census_data.empty:
        Median_Income_available = selected_row_in_census_data['Median Income'].iloc[0]
        #print("Median Income for block {}: {}".format(block, Median_Income_available))
    return Median_Income_available

In [51]:
def sq_ft_retrieval (house_id_to_find: str) -> int:
    sq_ft_available = 0
    #finds row the house_id is listed in the solar_csv
    selected_row_in_solar_data = solar_data[solar_data['house_id'] == house_id_to_find] 
    # if the row isn't empty we retrieve the house_id's sq_ft
    # The .iloc[0] is used to access the first row's sq_ft_available_for_solar_panels value from the selected_row 
    if not selected_row_in_solar_data.empty:
        sq_ft_available = selected_row_in_solar_data['sq_ft_available_for_solar_panels'].iloc[0]
        #print("sq_ft_available_for_solar_panels for house_id {}: {}".format(house_id_to_find, sq_ft_available))
    return sq_ft_available

In [52]:
def golden_ratio_block_list () -> list:
    list_of_blocks_with_golden_ratio_houses = {}
    # solar_house_ids_and_sqft = solar_data[['house_id', 'sq_ft_available_for_solar_panels']]
    # blocks_and_house_ids = census_data[['Block','Median Income']]

    # loop through for the length of the list of gas-houses
    for gas_house in range(len(gas_house_ids)): 
        #assigns house_id being used
        house_id_to_find = gas_house_ids[gas_house]
        
        Median_Income_available = median_income_retrieval(house_id_to_find)
        #print("Median Income for house {}: {}".format(house_id_to_find, median_income_retrival(house_id_to_find)))
        
        sq_ft_available = sq_ft_retrieval(house_id_to_find)
        #print("sq_ft_available_for_solar_panels for house_id {}: {}".format(house_id_to_find, sq_ft_retrieval(house_id_to_find)))

        if goldenratio(Median_Income_available, sq_ft_available, total_gas_usage_per_year):
            block = house_id_to_block.get(house_id_to_find)
            list_of_blocks_with_golden_ratio_houses.append(block)
    return list_of_blocks_with_golden_ratio_houses

golden_ratio_block_list()

{}